In [1]:
import os
import gc
import cv2
import random
import io

import fastai
from fastai.vision import *
from fastai.callbacks import *

from sklearn.model_selection import train_test_split, StratifiedKFold

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
train_df = pd.read_csv(os.path.join('..', 'input', 'train.csv'))

In [39]:
train_df.head()

,ImageId,ClassId,EncodedPixels,is_null,ClassId_is_null
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...,False,1
1,0002cc93b.jpg,2,NaN,True,0
2,0002cc93b.jpg,3,NaN,True,0
3,0002cc93b.jpg,4,NaN,True,0
4,00031f466.jpg,1,NaN,True,0


In [38]:
train_df['ClassId_is_null'] = train_df[['ClassId', 'is_null']].apply(lambda x: x['ClassId'] if not x['is_null'] else 0, axis=1)

In [55]:
train_idx, valid_idx, train_y, valid_y = train_test_split(train_df, train_df['ClassId_is_null'].values, test_size=0.2, random_state=43)

In [56]:
train_idx['ClassId_is_null'].value_counts()

0    34524
3     4130
1      710
4      650
2      203
Name: ClassId_is_null, dtype: int64

In [57]:
valid_idx['ClassId_is_null'].value_counts()

0    8653
3    1020
1     187
4     151
2      44
Name: ClassId_is_null, dtype: int64

In [51]:
train_df['ClassId_is_null'].value_counts()

0    43177
3     5150
1      897
4      801
2      247
Name: ClassId_is_null, dtype: int64

In [59]:
train_df.shape

(50272, 5)

In [60]:
train_df.to_csv('../input/train.csv', index=False)

In [64]:
train_df = train_df.pivot(index='ImageId',columns='ClassId',values='EncodedPixels')
train_df['defects'] = train_df.count(axis=1)

In [65]:
train_df.head()

ClassId,1,2,3,4,defects
ImageId,,,,,
0002cc93b.jpg,29102 12 29346 24 29602 24 29858 24 30114 24 3...,NaN,NaN,NaN,1
00031f466.jpg,NaN,NaN,NaN,NaN,0
000418bfc.jpg,NaN,NaN,NaN,NaN,0
000789191.jpg,NaN,NaN,NaN,NaN,0
0007a71bf.jpg,NaN,NaN,18661 28 18863 82 19091 110 19347 110 19603 11...,NaN,1


In [73]:
train_df[train_idx.index.values]

KeyError: '[27618 50116 31302 45414 ... 18448 40753 19776 14148] not in index'

In [71]:
train_idx

,ImageId,ClassId,EncodedPixels,is_null,ClassId_is_null
27618,8c26ebe1f.jpg,3,NaN,True,0
50116,ff1020719.jpg,1,NaN,True,0
31302,9f88a11c0.jpg,3,NaN,True,0
45414,e6a44cda3.jpg,3,134119 26 134324 77 134529 128 134734 179 1349...,False,3
49928,fe2136368.jpg,1,NaN,True,0
22972,74016c897.jpg,1,NaN,True,0
18466,5dd5484fc.jpg,3,218 39 474 39 730 39 985 40 1241 40 1497 40 17...,False,3
47226,efe30162b.jpg,3,62341 1 62465 1 62595 5 62721 2 62849 9 62977 ...,False,3
29004,9345f1250.jpg,1,NaN,True,0
7191,23f0398a4.jpg,4,NaN,True,0
